### 1. Organizar planilha com variáveis selecionadas

**G2 - Participação: Existência de Núcleos Comunitários de Proteção e Defesa Civil (Nupdec).**

Da planilha `munic_2020_gestao-de-riscos.csv`, selecionar a coluna:
- MGRD213

**G3 - Resposta: Existência de sistemas de alerta de riscos**. 

Da planilha `munic_2023_saneamento_drenagem.csv`, selecionar a coluna:
- SMAP126

**G4 - Reconhecimento: Existência de mapeamento e zoneamento de áreas de risco**. 

Da planilha `munic_2023_saneamento_drenagem.csv`, selecionar a coluna:
- SMAP122

**G5 - Reparação: Quantidade de políticas ou programas na área dos direitos humanos**. 

Da planilha `munic_2023_direitos-humanos.csv`, selecionar as coluna:
- MDHU571
- MDHU572
- MDHU573
- MDHU574
- MDHU575
- MDHU576
- MDHU577
- MDHU578
- MDHU579
- MDHU5710
- MDHU5711
- MDHU5712
- MDHU5713
- MDHU5714
- MDHU5715
- MDHU5716
- MDHU58
- MDHU61
- MDHU64
- MDHU67
- MDHU69

Salvar em data/raw/ibge/munic/t1 como `munic_selected_ijc.csv`

Manter as colunas `[cd_mun, sigla_uf, cd_uf, nm_mun, pop_mun, faixa_pop, regiao]` dos arquivos originais



In [ ]:
import pandas as pd
import os
import sys

sys.path.append(os.path.abspath('..'))
from src import config as cfg

# Caminho de saída automático
output_path = cfg.RAW_DIR / 'ibge' / 'munic' / 't1'
output_path.mkdir(parents=True, exist_ok=True)

cols_id = ['cd_mun', 'sigla_uf', 'cd_uf', 'nm_mun']

def carregar_e_selecionar(caminho, colunas_extras):
    try:
        df = pd.read_csv(caminho, sep=',', encoding='utf-8')
    except:
        df = pd.read_csv(caminho, sep=';', encoding='latin1')
    
    colunas_finais = cols_id + [c.lower() for c in colunas_extras]
    return df[colunas_finais]

# Base da pasta T0
base_t0 = cfg.RAW_DIR / 'ibge' / 'munic' / 't0'

# 1. G2 - Gestão de Riscos 2020
df_g2 = carregar_e_selecionar(
    base_t0 / '2020' / 'munic_2020_gestao-de-riscos.csv', 
    ['mgrd213']
)

# 2. G3 e G4 - Saneamento Drenagem 2023
df_g3_g4 = carregar_e_selecionar(
    base_t0 / '2023_saneamento' / 'munic_2023_saneamento_drenagem.csv', 
    ['smap126', 'smap122']
)

# 3. G5 - Direitos Humanos 2023
df_g5 = carregar_e_selecionar(
    base_t0 / '2023' / 'munic_2023_direitos-humanos.csv', 
    [
    'mdhu571', 'mdhu572', 'mdhu573', 'mdhu574', 'mdhu575', 'mdhu576', 
    'mdhu577', 'mdhu578', 'mdhu579', 'mdhu5710', 'mdhu5711', 'mdhu5712', 
    'mdhu5713', 'mdhu5714', 'mdhu5715', 'mdhu5716', 'mdhu58', 'mdhu61', 
    'mdhu64', 'mdhu67', 'mdhu69'
    ]
)

df_final = df_g2.merge(df_g3_g4, on=cols_id, how='left')
df_final = df_final.merge(df_g5, on=cols_id, how='left')

# Salvar
file_out = output_path / 'munic_selected_ijc.csv'
df_final.to_csv(file_out, index=False, encoding='utf-8-sig')

print(f"Arquivo salvo com sucesso em: {file_out}")
print(f"Total de colunas: {len(df_final.columns)}")

Arquivo salvo com sucesso em: ../data/raw/ibge/munic/selected/munic_selected_ijc.csv
Total de colunas: 28


### 2. Limpar arquivo
Manter apenas os valores True, False e NaN


In [ ]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.abspath('..'))
from src import config as cfg

file_path = cfg.RAW_DIR / 'ibge' / 'munic' / 't1' / 'munic_selected_ijc.csv'
df = pd.read_csv(file_path)

# Lista das colunas que serão limpas
colunas_interesse = [
    'mgrd213', 'smap126', 'smap122', 'mdhu571', 'mdhu572', 'mdhu573', 
    'mdhu574', 'mdhu575', 'mdhu576', 'mdhu577', 'mdhu578', 'mdhu579', 
    'mdhu5710', 'mdhu5711', 'mdhu5712', 'mdhu5713', 'mdhu5714', 
    'mdhu5715', 'mdhu5716', 'mdhu58', 'mdhu61', 'mdhu64', 'mdhu67', 'mdhu69'
]

# Definimos o mapeamento
# Note que qualquer valor que NÃO esteja no dicionário ficará como nulo (NaN) ao usar o map
mapa_bool = {
    'Sim': True,
    'Não': False
}

# Aplicar a limpeza
for col in colunas_interesse:
    if col in df.columns:
        # Usamos .map() com o dicionário. Valores não encontrados viram NaN.
        df[col] = df[col].map(mapa_bool)

# Salvar o novo arquivo
output_clean = cfg.RAW_DIR / 'ibge' / 'munic' / 't1' / 'munic_selected_ijc_clean.csv'
df.to_csv(output_clean, index=False, encoding='utf-8-sig')

print(f"Limpeza concluída! Arquivo salvo em: {output_clean}")
print("\nExemplo de como ficaram os valores na coluna MGRD213:")
print(df['mgrd213'].value_counts(dropna=False))

Limpeza concluída! Arquivo salvo em: ../data/raw/ibge/munic/selected/munic_selected_ijc_clean.csv

Exemplo de como ficaram os valores na coluna MGRD213:
mgrd213
False    5084
True      377
NaN       109
Name: count, dtype: int64


### 3. Organizar arquivos csv finais

A partir do arquivo `data/raw/ibge/munic/t1/munic_selected_ijc_clean.csv`:

**G2 - Participação: Existência de Núcleos Comunitários de Proteção e Defesa Civil (Nupdec)**

Salvar em `data/clean/g2_mun_nupdec.csv` com a coluna `g2_par_norm` para aqueles valores de `mgrd213` em que 1 = True e 0 = False. Nulos são mantidos como nulos. Manter coluna `cd_mun`.

**G3 - Resposta - Existência de sistemas de alerta de riscos**

Salvar em `data/clean/g3_mun_alerta.csv` com a coluna `g3_res_norm` para aqueles valores de `smap126` em que 1 = True e 0 = False. Nulos são mantidos como nulos. Manter coluna `cd_mun`.

**G4 - Reconhecimento - Existência de mapeamento e zoneamento de áreas de risco**

Salvar em `data/clean/g4_mun_mapeamento.csv` com a coluna `g4_rec_norm` para aqueles valores de `smap122` em que 1 = True e 0 = False. Nulos são mantidos como nulos. Manter coluna `cd_mun`.

**G5 - Assistência: Quantidade de políticas ou programas na área dos direitos humanos**

Salvar em `data/clean/g5_mun_politicas_direitos_humanos.csv` com a coluna `g5_ass_abs` com a quantidade de valores `True` contidos nas colunas [mdhu571, mdhu572, mdhu573, mdhu574, mdhu575, mdhu576, mdhu577, mdhu578, mdhu579, mdhu5710, mdhu5711, mdhu5712, mdhu5713, mdhu5714, mdhu5715, mdhu5716, mdhu58, mdhu61, mdhu64, mdhu67, mdhu69]. Se todos os valores forem nulos, deve manter o valor nulo. Deve ainda criar a coluna `g5_ass_norm` com o valor normalizado de 0 a 1, onde 0 = menor quantidade e 1 = maior quantidade.

In [ ]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.abspath('..'))
from src import config as cfg

# Lê de t1
input_file = cfg.RAW_DIR / 'ibge' / 'munic' / 't1' / 'munic_selected_ijc_clean.csv'
df = pd.read_csv(input_file)

# Cria e configura o caminho de saída para t2
output_dir = cfg.RAW_DIR / 'ibge' / 'munic' / 't2'
output_dir.mkdir(parents=True, exist_ok=True)

def bool_to_numeric(series):
    return series.map({True: 1, False: 0})

# G2: Nupdec
g2 = df[['cd_mun', 'mgrd213']].copy()
g2['g2_par_norm'] = bool_to_numeric(g2['mgrd213'])
g2[['cd_mun', 'g2_par_norm']].to_csv(output_dir / 'g2_mun_nupdec.csv', index=False)

# G3: Alerta
g3 = df[['cd_mun', 'smap126']].copy()
g3['g3_res_norm'] = bool_to_numeric(g3['smap126'])
g3[['cd_mun', 'g3_res_norm']].to_csv(output_dir / 'g3_mun_alerta.csv', index=False)

# G4: Mapeamento
g4 = df[['cd_mun', 'smap122']].copy()
g4['g4_rec_norm'] = bool_to_numeric(g4['smap122'])
g4[['cd_mun', 'g4_rec_norm']].to_csv(output_dir / 'g4_mun_mapeamento.csv', index=False)

# G5: Direitos Humanos
cols_g5 = [
    'mdhu571', 'mdhu572', 'mdhu573', 'mdhu574', 'mdhu575', 'mdhu576', 
    'mdhu577', 'mdhu578', 'mdhu579', 'mdhu5710', 'mdhu5711', 'mdhu5712', 
    'mdhu5713', 'mdhu5714', 'mdhu5715', 'mdhu5716', 'mdhu58', 'mdhu61', 
    'mdhu64', 'mdhu67', 'mdhu69'
]

g5_data = df[['cd_mun'] + cols_g5].copy()
g5_data['g5_rep_abs'] = g5_data[cols_g5].sum(axis=1, min_count=1)

min_val = g5_data['g5_rep_abs'].min()
max_val = g5_data['g5_rep_abs'].max()
g5_data['g5_rep_norm'] = (g5_data['g5_rep_abs'] - min_val) / (max_val - min_val)

g5_data[['cd_mun', 'g5_rep_abs', 'g5_rep_norm']].to_csv(output_dir / 'g5_mun_politicas_direitos_humanos.csv', index=False)

print(f"Arquivos salvos com sucesso na pasta: {output_dir}")

Arquivos salvos com sucesso em 'data/clean/'


### 4. Gerar diagnóstico dos resultados nos arquivos

In [ ]:
import pandas as pd
import os
import sys

# 1. Adiciona a raiz do projeto e importa o config
sys.path.append(os.path.abspath('..'))
from src import config as cfg

# Caminho apontando para a pasta t2
input_dir = cfg.RAW_DIR / 'ibge' / 'munic' / 't2'

# Dicionário com os caminhos corretos
arquivos = {
    'G2 (Nupdec)': input_dir / 'g2_mun_participacao_nupdec.csv',
    'G3 (Alerta)': input_dir / 'g3_mun_resposta_alerta.csv',
    'G4 (Mapeamento)': input_dir / 'g4_mun_reconhecimento_mapeamento.csv',
    'G5 (Direitos Humanos)': input_dir / 'g5_mun_reparacao_direitos.csv'
}

print("=== DIAGNÓSTICO DOS ARQUIVOS FINAIS ===\n")

for nome, path in arquivos.items():
    if path.exists():
        df = pd.read_csv(path)
        print(f"--- {nome} ---")
        
        # Identificar a coluna de valor (a que não é cd_mun)
        col_valor = [c for c in df.columns if c != 'cd_mun']
        
        for col in col_valor:
            nulls = df[col].isna().sum()
            
            # Diagnóstico para as colunas normalizadas (0 e 1)
            if 'norm' in col:
                ones = (df[col] == 1).sum()
                zeros = (df[col] == 0).sum()
                print(f"Coluna: {col}")
                print(f"  > Valor 1 (Sim/Máximo): {ones}")
                print(f"  > Valor 0 (Não/Mínimo): {zeros}")
                print(f"  > Valores Nulos (NaN): {nulls}")
            
            # Diagnóstico extra para o G5 (Valores Absolutos)
            if col == 'g5_ass_abs':
                print(f"Coluna: {col}")
                print(f"  > Média de políticas: {df[col].mean():.2f}")
                print(f"  > Mediana: {df[col].median()}")
                print(f"  > Máximo de políticas encontradas: {df[col].max()}")
                print(f"  > Mínimo de políticas encontradas: {df[col].min()}")
                print(f"  > Distribuição (Quartis):\n{df[col].describe(percentiles=[.25, .5, .75])[['25%', '50%', '75%']]}")
                
        print("-" * 40)
    else:
        print(f"⚠️ Arquivo não encontrado: {path.name}\n" + "-" * 40)

=== DIAGNÓSTICO DOS ARQUIVOS FINAIS ===

--- G2 (Nupdec) ---
Coluna: g2_par_norm
  > Valor 1 (Sim/Máximo): 377
  > Valor 0 (Não/Mínimo): 5084
  > Valores Nulos (NaN): 109
----------------------------------------
--- G3 (Alerta) ---
Coluna: g3_res_norm
  > Valor 1 (Sim/Máximo): 613
  > Valor 0 (Não/Mínimo): 2363
  > Valores Nulos (NaN): 2594
----------------------------------------
--- G4 (Mapeamento) ---
Coluna: g4_rec_norm
  > Valor 1 (Sim/Máximo): 2327
  > Valor 0 (Não/Mínimo): 649
  > Valores Nulos (NaN): 2594
----------------------------------------
--- G5 (Direitos Humanos) ---
Coluna: g5_ass_abs
  > Média de políticas: 6.96
  > Mediana: 6.0
  > Máximo de políticas encontradas: 20.0
  > Mínimo de políticas encontradas: 1.0
  > Distribuição (Quartis):
25%    5.0
50%    6.0
75%    9.0
Name: g5_ass_abs, dtype: float64
Coluna: g5_ass_norm
  > Valor 1 (Sim/Máximo): 14
  > Valor 0 (Não/Mínimo): 4
  > Valores Nulos (NaN): 9
----------------------------------------
